In [ ]:
# Install HINGE

# Also need to install pbcore (other dependencies?)

In [ ]:
# Download data for NCTC9657

In [26]:
cd /Users/ilanshom/Documents/AwesomeAssembler/

/Users/ilanshom/Documents/AwesomeAssembler


In [29]:
!source utils/setup.sh

In [31]:
cd /Users/ilanshom/Documents/HINGE-analyses/NCTC9657/

/Users/ilanshom/Documents/HINGE-analyses/NCTC9657


In [33]:
!bash alignment.sh NCTC9657

alignment.sh: line 2: correct_head.py: command not found
alignment.sh: line 4: fasta2DB: command not found
alignment.sh: line 6: DBsplit: command not found
alignment.sh: line 8: HPCdaligner: command not found
rm: NCTC9657.*.NCTC9657.*.las: No such file or directory
alignment.sh: line 12: LAmerge: command not found
alignment.sh: line 14: DASqv: command not found
